In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [6]:
roll_up = SparkSession.builder.appName("Rolling_Windows").getOrCreate()
df_spark = roll_up.read.csv("train.csv", header=True, inferSchema=True)

25/11/24 16:35:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


**Rolling 7-Day Average**
(Used in forecasting models like ARIMA, XGBoost, Prophet, etc.)

In [7]:
windowSpec = Window.partitionBy('Store').orderBy('Date').rowsBetween(-6,0)
df_spark.withColumn('7_Day_Rolling_Avg', avg('Sales').over(windowSpec)).show()

+-----+---------+----------+-----+---------+----+-----+------------+-------------+------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday| 7_Day_Rolling_Avg|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|               0.0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|            2479.0|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|            3415.0|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|            3931.5|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|            3559.2|
|   28|        7|2013-01-06|    0|        0|   0|    0|           a|            0|            2966.0|
|   28|        1|2013-01-07|10126|      989|   1|    1|           0|            0|

In [9]:
df_spark.withColumn('Sales_7Day_Avg', avg(col('Sales')).over(windowSpec)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_7Day_Avg|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|           0.0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|        2479.0|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|        3415.0|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|        3931.5|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|        3559.2|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
only showing top 5 rows



In [13]:
sumwindowSpec = Window.partitionBy('Store').orderBy('Date').rowsBetween(-13,0)
df_spark.withColumn('Sales_3Day_Sum', sum(col('Sales')).over(sumwindowSpec)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_3Day_Sum|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|             0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|          4958|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|         10245|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|         15726|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|         17796|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
only showing top 5 rows



In [14]:
maxwindowSpec = Window.partitionBy('Store').orderBy('Date').rowsBetween(-2,0)
df_spark.withColumn('Sales_3Day_MAx', max(col('Sales')).over(maxwindowSpec)).show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Sales_3Day_MAx|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|             0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|          4958|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|          5287|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|          5481|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|          5481|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+
only showing top 5 rows



25/11/25 02:10:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1008608 ms exceeds timeout 120000 ms
25/11/25 02:10:54 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/25 02:28:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 